# Introducción

Este documento describe cómo partir de un sistema Debian 10 recién instalado y desplegar una aplicación web basada en Django con una robustez propia de sistemas en producción.

# Prerrequisitos

### Gestión de `sudo`

En una instalación por defecto de debian, el comando `sudo` no está disponible, por lo que hay que instalarlo primero convirtiéndonos en usuario _root_; a continuación hay que añadir a nuestro usuario no privilegiado al grupo `sudo` para que pueda disfrutar de su uso:

```bash
su -   # aquí pedirá la contraseña de *administrador*
apt update && apt install sudo
usermod -aG sudo <USUARIO> # sustituir <USUARIO> por el creado durante la instalación
exit   # abandonamos la sesión de root
```

De nuevo en la sesión del usuario normal, aplicamos los cambios en los grupos:

```bash
newgrp sudo
```

### Instalación de paquetes

Nos van a hacer falta los siguientes paquetes:

```bash
sudo apt update && sudo apt install -y \
    git \
    nginx \
    python3-pip \
    python3-venv
```

# Preparación del proyecto

### Pasos preliminares

Para asegurarnos de que no nos confundimos con el directorio en el que estamos trabajando, crearemos una variable de entorno `SHIELD_DIR` que apunte a un directorio llamado `shield` en el _home_ del usuario con el que estemos trabajando. Por ejemplo, si el usuario dentro de la máquina virtual se llama `debian`, entonces `SHIELD_DIR` será equivalente a `/home/debian/shield`:

```bash
export SHIELD_DIR="$HOME/shield"
```

### Clonar desde GitHub

Es importante recordar que hay que ajustar la URL que usa `git clone` para que corresponda con la del proyecto personal de cada uno, tal que así:

```bash
git clone https://github.com/<USER>/<REPO>.git "$SHIELD_DIR"
```

Por ejemplo, en mi caso el comando correcto sería:

```bash
git clone https://github.com/asamarin/shield.git "$SHIELD_DIR"
```

### Creación de entorno virtual

Dentro del directorio del proyecto, creamos el entorno virtual de Python en el que instalaremos las dependencias que necesitamos para ejecutar correctamente la aplicación web:

```bash
python3 -m venv "$SHIELD_DIR/venv"
```

Seguidamente, lo activamos:

```bash
source "$SHIELD_DIR/venv/bin/activate"
```

### Instalación de dependencias

A la hora de instalar dependencias con `pip3`, y tal y como ocurrió varias veces en clase, a veces podemos encontrarnos un error tal que así:

`error: invalid command 'bdist_wheel'`

Para solucionarlo, es tan sencillo como asegurarnos que tenemos el módulo `wheel` presente en el entorno virtual:

```bash
pip3 install wheel
```

Ahora sí, toca instalar las dependencias del proyecto como tal. En nuestro caso ya están especificadas en el fichero `requirements.txt`, por tanto nos limitamos a indicarle a `pip3` que instale lo que allí encuentre:

```bash
pip3 install -r "$SHIELD_DIR/requirements.txt"
```

### Migraciones y carga de datos en Django

Para que nuestra aplicación no parezca vacía, tenemos que aplicar las migraciones pertinentes y cargar los datos presentes en el fichero `$SHIELD_DIR/metahumans/dumpdata.json`:

```bash
python3 "$SHIELD_DIR/manage.py" makemigrations
python3 "$SHIELD_DIR/manage.py" migrate
python3 "$SHIELD_DIR/manage.py" loaddata "$SHIELD_DIR/metahumans/dumpdata.json"
```

Recolectamos también todos los _assets_ estáticos (imágenes, ficheros HTML, CSS, Javascript, etc.) en un nuevo directorio llamado _static_. Esto es una optimización para que `nginx` se encargue de servir directamente este tipo de ficheros que no requieren de ninguna intervención por parte de Django, liberando por tanto de carga de trabajo a este último:

```bash
echo "STATIC_ROOT = os.path.join(BASE_DIR, 'static')" >> "$SHIELD_DIR/shield/settings.py"
python3 "$SHIELD_DIR/manage.py" collectstatic
```

### Mover proyecto a su localización final

Una vez que hemos preparado el proyecto con todo lo necesario para ser ejecutado (virtualenv, dependencias, base de datos cargada, etc.), es el momento de trasladar nuestro proyecto a una localización más adecuada en nuestro sistema de ficheros. Ahora mismo, todo está en el _home_ del usuario con el que estamos trabajando en la máquina virtual, pero esto no es óptimo; lo ideal es que usuarios "reales" del sistema no tengan que estar exponiendo parte de sus ficheros personales a otros usuarios del sistema, aunque sean usuarios "robóticos" como los que ejecutan los servicios encargados de manejar tráfico HTTP (`nginx` y `uWSGI`). Utilizaremos la localización que `nginx` nos dispone por defecto, que es `/var/www` en Ubuntu/Debian:

```bash
deactivate # salimos del venv ya que lo vamos a trasladar
sudo mv "$SHIELD_DIR" /var/www
```

Nuestro nuevo directorio de SHIELD será, por tanto, `/var/www/shield`, así que lo que haremos a continuación será actualizar la variable de entorno que estábamos utilizando:

```bash
export SHIELD_DIR="/var/www/shield"
```

A partir de este momento, `$SHIELD_DIR` referirá al directorio `/var/www/shield`.

Una pregunta válida en este punto es: ¿por qué no lo hicimos todo en `/var/www/shield` desde que clonamos el proyecto? Es totalmente posible, pero de este modo nos ahorramos usar `sudo` con frecuencia, ya que cualquiera de los comandos que hemos ejecutado anteriormente que quisiese añadir/modificar/borrar ficheros hubiera necesitado permisos de `root`.

### Ajustar permisos

Tan pronto como es posible después de la inicialización, `nginx` crea nuevos procesos hijos (los _workers_) para hacer el trabajo real de servir contenido web usando el protocolo HTTP. Es importante que estos procesos hijos tengan privilegios reducidos—principalmente para reducir el impacto de un hipotético agujero de seguridad en el worker—por lo que por defecto el usuario (`uid`) que utiliza `nginx` para dichos workers es `www-data`. Cambiamos tanto el usuario como el grupo dueños del directorio `www-data` para que `nginx` no tenga problemas de permisos a la hora de acceder aquí:

```bash
sudo chown -R www-data:www-data "$SHIELD_DIR"
```

# Configuración de `uWSGI`

Con `uWSGI` pretendemos seguir el patrón "emperador/vasallos": Existe un proceso maestro (el _emperador_) que se encarga de monitorizar a cada uno de sus procesos hijos (los _vasallos_) y sus correspondientes ficheros de configuración, matando y re-iniciando a los vasallos tan pronto como haya cambios dichos ficheros. Lo normal es tener un vasallo por cada sitio web hospedado en la máquina, por lo que en nuestro caso sólo tendremos un único vasallo, el correspondiente a nuestra aplicación web de SHIELD.

### Instalación _system-wide_

Existen dos enfoques igualmente sencillos para la instalación de `uwsgi`: bien usando `pip3`, o bien usando `apt`. La que vimos en clase es la primera, pero la segunda es igualmente válida; de hecho, la configuración posterior es bastante más sencilla. Vamos a describir los pasos que seguimos en clase, por lo que optaremos por la opción de `pip3`:

```bash
sudo pip3 install uwsgi
```

Y procedemos a crear la jerarquía de directorios que nos será necesaria en `/etc` para alojar los ficheros de configuración:

```bash
sudo mkdir -p /etc/uwsgi/vassals
```

Como recordatorio, la opción `-p` de `mkdir` significa "crea este directorio aún en el caso de que alguno de los directorios intermedios en la ruta no exista".

### Configuración del vasallo

Primero, creamos el fichero de configuración del vasallo (este fichero puede estar también en el repositorio de GitHub, de forma que no sería necesario crearlo aquí):

```bash
sudo tee "/var/www/shield/uwsgi.ini" &>/dev/null <<EOF
[uwsgi]
# Django-related settings
# the base directory (full path)
chdir           = /var/www/shield
# Django's wsgi file
module          = shield.wsgi
# the virtualenv (full path)
home            = /var/www/shield/venv

# process-related settings
# maximum number of worker processes
processes       = 2
# the socket (use the full path to be safe
socket          = /tmp/shield.sock
# ... with appropriate permissions - may be needed
chmod-socket    = 660
chown-socket    = www-data:www-data
uid             = www-data
gid             = www-data
# clear environment on exit
vacuum          = true
logto           = /var/log/uwsgi/shield.log
EOF
```

Con respecto a esa última directiva `logto`, recordemos que hay que crear el fichero de antemano y asignarle los permisos adecuados para que el vasallo pueda escribir en él; será suficiente con cambiar el usuario a `www-data`:

```bash
sudo mkdir -p /var/log/uwsgi
sudo touch /var/log/uwsgi/shield.log
sudo chown www-data /var/log/uwsgi/shield.log
```

Ahora creamos el enlace simbólico:

```bash
sudo ln -s /var/www/shield/uwsgi.ini /etc/uwsgi/vassals/shield.ini
```

### Configuración del emperador

Misma lógica que antes, esta vez para crear el fichero `/etc/uwsgi/emperor.ini`:

```bash
sudo tee "/etc/uwsgi/emperor.ini" &>/dev/null <<EOF
[uwsgi]
emperor = /etc/uwsgi/vassals
uid = www-data
gid = www-data
EOF
```

Ahora creamos el servicio de `systemd` para gestionar el ciclo de vida del emperador:

```bash
sudo tee "/etc/systemd/system/uwsgi-emperor.service" &>/dev/null <<EOF
[Unit]
Description=uWSGI Emperor
After=syslog.target

[Service]
ExecStart=/usr/local/bin/uwsgi --ini /etc/uwsgi/emperor.ini
# Requires systemd version 211 or newer
RuntimeDirectory=uwsgi
Restart=always
KillSignal=SIGQUIT
Type=notify
StandardError=syslog
NotifyAccess=all

[Install]
WantedBy=multi-user.target
EOF
```

Cuando añadimos un nuevo servicio, conviene informar de ello a `systemd` para que relea todos los ficheros de configuración disponibles, incluyendo el que acabamos de crear:

```bash
sudo systemctl daemon-reload
```

Ahora podremos arrancar nuestro nuevo servicio:

```bash
sudo systemctl start uwsgi-emperor
```

Comprobamos brevemente que todo está en orden; esto podemos hacerlo con:

```bash
sudo systemctl status uwsgi-emperor
```

Algunas de las pistas que podemos buscar para estar seguros de que realmente se ha iniciado correctamente se pueden ver resaltadas en la siguiente imagen:

![Emperor service status](./uwsgi-emperor-status.png)

Una vez comprobado esto, sólo nos queda configurar el emperador para que se inicie automáticamente cada vez que la máquina se reinicia:

```bash
sudo systemctl enable uwsgi-emperor
```

# Configuración de `nginx`

El último eslabón de la cadena es `nginx`, nuestro servidor web. Es el encargado de recibir y procesar todas las peticiones HTTP que vienen desde el exterior, así como de transmitir ficheros estáticos y las respuestas dinámicas generadas por nuestra aplicación, de la manera más eficiente posible. Empezaremos por desactivar el sitio web por defecto (el manido _"Welcome to nginx!"_):

```bash
sudo rm /etc/nginx/sites-enabled/default
```

Ahora creamos el fichero de configuración para nuestro sitio:

```bash
sudo tee "/etc/nginx/sites-available/shield.conf" &>/dev/null <<EOF
# the upstream component nginx needs to connect to
upstream django {
    server unix:///tmp/shield.sock;
}

# configuration of the server
server {
    # the port your site will be served on
    listen      80;
    # the domain name it will serve for
    server_name localhost; # substitute your machine's IP address or FQDN
    charset     utf-8;

    location /static {
        alias /var/www/shield/static; # your Django project's static files - amend as required
    }

    # Finally, send all non-media requests to the Django server.
    location / {
        uwsgi_pass  django;
        include     /etc/nginx/uwsgi_params; # the uwsgi_params file you installed
    }
}
EOF
```

No debemos olvidarnos de crear un nuevo enlace simbólico apuntando desde `sites-enabled` (los que están activos ahora mismo) a `sites-available` (los que están disponibles, aunque no necesariamente funcionando):

```bash
sudo ln -s /etc/nginx/sites-available/shield.conf /etc/nginx/sites-enabled/shield.conf
```

Ahora solo queda notificar a `nginx` de los cambios:

```bash
sudo systemctl restart nginx
```

Y probar, bien con el navegador o bien desde la propia consola:

```bash
wget localhost -qO-
```

Si la respuesta recibida parece HTML (y menciona SHIELD por alguna parte), lo hemos conseguido.